# **Padrões de vendas de jogos digitais** 🎮 

José Henrique de Oliveira<br>
[Portfólio](https://joseportfolio.notion.site/Portf-lio-do-Z-308350bb4f4544adb87da2a1a83b8f78) | [LinkedIn](https://www.linkedin.com/in/jholiveira94/)

---

# **Introdução**

A industria de jogos digitais entrega grande volume de produções todos os anos e, como qualquer outro setor, necessita que seus produtos **desempenhem bem no mercado**.<br>
Portanto, **determinar quais nichos de jogos e clientes** trarão tal resultado é essencial para a continuidade dos projetos de uma empresa desse ramo.

Com esse contexto em mente, durante este estudo serão analisadas as vendas de jogos digitais com objetivo de encontrar padrões que auxiliem nessa tomada de decisão tão impactante.<br>
O conjunto de dados escolhido para conduzir essa investigação pode ser encontrado na plataforma Kaggle: [**Video Game Sales**](https://www.kaggle.com/datasets/gregorut/videogamesales/data)

### **Problemas de negócio**

Ao longo desse trabalho, será construído um **modelo de recomendação de lançamento de jogos por região** baseado nos padrões de vendas observados nos dados.<br>
Para tal, serão respondidas questões mais objetivas que auxiliarão a destrinchar o tema elencado.<br>
As perguntas estão agrupadas para facilitar a organização geral do estudo.

##### **Objetivo**

<div style="background-color: purple; color: white; padding: 10px; border-radius: 5px;">
<b>Recomendação de gêneros populares por região</b> <br>
Quais gêneros de jogos devem ser lançados em uma região específica para maximizar as vendas?
</div>

##### **Questões norteadoras**
Análise das vendas globais por gênero e plataforma:
- Quais gêneros de jogos têm o maior sucesso em termos de vendas globais?
- Como as vendas variam entre diferentes plataformas para um mesmo gênero de jogo?

Distribuição geográfica das vendas:
- Qual região (América do Norte, Europa, Japão, Outras) é mais forte em termos de vendas para diferentes gêneros ou plataformas?
- Existem padrões regionais nas preferências de gêneros de jogos?

Tendências temporais de lançamento e vendas:
- Como as vendas de jogos mudaram ao longo do tempo?
- Existe uma correlação entre o ano de lançamento e o sucesso de vendas em diferentes regiões?

Impacto da publisher nas vendas:
- Quais publishers têm consistentemente os jogos mais vendidos?
- Existe uma relação entre o publisher e o sucesso regional ou global de um jogo?

Comparação de sucesso entre plataformas:
- Quais plataformas tendem a ter jogos com maiores vendas globais e por região?
- Existe uma diferença significativa nas vendas entre plataformas para jogos do mesmo gênero?

---

# **1. Preparações**

### **1.1 Bibliotecas**

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

### **1.2 Carregando e conhecendo os dados**

In [3]:
df = pd.read_csv('vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Nesse conjunto de dados, há os seguintes atributos:

- Rank - Classificação geral das vendas
- Name - Nome do jogo
- Platform - Platforma de lançamento do jogo (PC,PS4, etc.)
- Year - Ano de lançamento do jogo
- Genre - Gênero do jogo
- Publisher - Empresa responsável pela publicação do jogo
- NA_Sales - Vendas na América do Norte (em milhões)
- EU_Sales - Vendas na Europa (em milhões)
- JP_Sales - Vendas no Japão (em milhões)
- Other_Sales - Vendas no restante do mundo (em milhões)
- Global_Sales - Total mundial de vendas

Considerando que este estudo está em português, algumas expressões serão traduzidas para melhor localização do estudo.<br>
Segue relação de tradução:

|   **Original**   |    **Tradução**    |
| :----------: | :------------: |
|     Rank     |       -        |
|     Name     |      Nome      |
|   Platform   |   Plataforma   |
|     Year     |      Ano       |
|    Genre     |     Genero     |
|  Publisher   |       -        |
|   NA_Sales   |   Vendas_AN    |
|   EU_Sales   |   Vendas_EU    |
|   JP_Sales   |   Vendas_JP    |
| Other_Sales  | Vendas_Outras  |
| Global_Sales | Vendas_Globais |

As expressões **Rank** e **Platform** não receberão tradução pelo uso comum da comunidade de jogos, mesmo no Brasil.

### **1.3 Adequações iniciais**

In [4]:
'''
Considerando que o Rank traz informação individualizada de cada jogo e esse estudo focará em comportamentos agrupados,
a coluna Rank será retirada. As demais permanecerão.
'''

df = df.drop('Rank', axis = 1)

In [5]:
''' 
Como colocado na Contextualização, as colunas serão traduzidas.
'''

df.columns = ['Nome', 'Plataforma', 'Ano', 'Genero', 'Publisher', 'Vendas_AN', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras', 'Vendas_Globais']

In [6]:
''' 
Verificando a tipagem das features e a existência de valores nulos
'''

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nome            16598 non-null  object 
 1   Plataforma      16598 non-null  object 
 2   Ano             16327 non-null  float64
 3   Genero          16598 non-null  object 
 4   Publisher       16540 non-null  object 
 5   Vendas_AN       16598 non-null  float64
 6   Vendas_EU       16598 non-null  float64
 7   Vendas_JP       16598 non-null  float64
 8   Vendas_Outras   16598 non-null  float64
 9   Vendas_Globais  16598 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.3+ MB


<div style="background-color: purple; color: white; padding: 10px; border-radius: 5px;">
💡 Percepções
</div>

A partir dessas informações, conclui-se que:
- A tipagem da feature Ano pode ser adequada para int.
- Há valores nulos nas features Ano e Publisher. 
    - Já pode-se entender que é uma quantidade insignificante, portanto não atrapalhará o estudo.

In [7]:
''' 
Modificando a tipagem da coluna Ano.
Para isso, os valores NA serão preenchidos por zero.
'''

df['Ano'] = df.Ano.fillna(0)

df['Ano'] = df['Ano'].astype(int)

In [8]:
''' 
Verificando as alterações
'''

df['Ano'].info()

df.head(3)

<class 'pandas.core.series.Series'>
RangeIndex: 16598 entries, 0 to 16597
Series name: Ano
Non-Null Count  Dtype
--------------  -----
16598 non-null  int32
dtypes: int32(1)
memory usage: 65.0 KB


,Nome,Plataforma,Ano,Genero,Publisher,Vendas_AN,Vendas_EU,Vendas_JP,Vendas_Outras,Vendas_Globais
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82


In [22]:
''' 
Verificação das plataformas, gêneros e publishers apresentadas nesse conjunto de dados
'''

for c in df[['Plataforma', 'Genero', 'Publisher']]:
    if c in df[['Plataforma', 'Genero']]:
        print(f'A quantidade de valores únicos de {c} é:', df[c].nunique(), '\n')
        print(f'Os valores únicos de {c} são:\n', df[c].unique(), '\n')
        print('-'*85)
    else:
        print(f'A quantidade de valores únicos de {c} é:', df[c].nunique())
        print(f'Em função da quantidade, optou-se por suprimir a lista de valores únicos de {c}.')

A quantidade de valores únicos de Plataforma é: 31 

Os valores únicos de Plataforma são:
 ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX'] 

-------------------------------------------------------------------------------------
A quantidade de valores únicos de Genero é: 12 

Os valores únicos de Genero são:
 ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy'] 

-------------------------------------------------------------------------------------
A quantidade de valores únicos de Publisher é: 578
Em função da quantidade, optou-se por suprimir a lista de valores únicos de Publisher.


<div style="background-color: purple; color: white; padding: 10px; border-radius: 5px;">
💡 Percepções
</div>

A verificação de valores únicos auxilia para compreender a diversidade das informações presentes no conjunto de dados.<br>
Por exemplo, fica nítido que uma análise individualizada por Plataforma, Gênero ou Publisher é inviável se não houver seleção ou agrupamento desses dados.

Por outro lado, fortalece as questões norteadoras quanto ao estudo de comportamentos generalizados e quanto a buscar valores com maior impacto.

In [ ]:
''' 
Verificação geral de estatísticas descritivas do Dataset
'''

df.describe().round(2)

,Ano,Vendas_AN,Vendas_EU,Vendas_JP,Vendas_Outras,Vendas_Globais
count,16598.00,16598.00,16598.00,16598.00,16598.00,16598.00
mean,1973.65,0.26,0.15,0.08,0.05,0.54
std,254.35,0.82,0.51,0.31,0.19,1.56
min,0.00,0.00,0.00,0.00,0.00,0.01
25%,2003.00,0.00,0.00,0.00,0.00,0.06
50%,2007.00,0.08,0.02,0.00,0.01,0.17
75%,2010.00,0.24,0.11,0.04,0.04,0.47
max,2020.00,41.49,29.02,10.22,10.57,82.74


<div style="background-color: purple; color: white; padding: 10px; border-radius: 5px;">
💡 Percepções
</div>

Considerando que as vendas estão na casa dos milhões, como descrito no **Contexto**, percebe-se com essa análise que a maioria dos jogos não atinge quantidade expressiva de vendas.

Apenas a partir do terceiro quartil encontram-se jogos com ao menos um milhão de cópias vendidas, mesmo olhando para Vendas_Globais, que é a soma das regiões.

### **1.4 Considerações iniciais**

A partir das verificações feitas nesse tópico, constata-se que:
- As questões norteadoras previamente elencadas são coerentes para esse conjunto de dados.
- Analisar comportamentos a partir de agrupamentos, como por região ou gênero, será importante para que dados menos expressivos tenham mais espaço no estudo.

---

# **2. EDA - Análise Exploratória dos Dados**

Nessa etapa, serão respondidas as 

Análise das vendas globais por gênero e plataforma